In [4]:
# !pip install camelot-py
# !pip install opencv-python
# !pip install ghostscript
# !pip install pdfplumber

## Text Extraction

In [158]:
import pandas as pd
import pdfplumber
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#pdf = pdfplumber.open("/Users/angshuman.chakraborty@cognitedata.com/Desktop/work/BP/DDRs_samples_BP_GoM/2017/83_782_16-Daily_Ops-Partners-Report_83-(07-10-2017).pdf")

col_list = ["From - To\nOp. Depth (usft)", "Hrs\n(hr)","Phase",'Task', 'Activity', 'Code', 'NPT', 'Critical \nPath', 'Operation']
def columns_check(df):
    ll = []
    for a in df.columns:
        if a.startswith(col_list[0]) | a.startswith(col_list[1]) | a.startswith(col_list[2]) | a.startswith(col_list[3]) | a.startswith(col_list[4]) | a.startswith(col_list[5]) | a.startswith(col_list[6]) | a.startswith(col_list[7]) | a.startswith(col_list[8]):
            ll.append(a)
    return df[ll]


def list_dir(root):
    result = []
    
    for path, dirs, files in os.walk(root):
        for name in files:
            if name.lower().endswith((".pdf")):
                result.append(os.path.join(path, name))
                
    return result


def check_df_1(DF):
    for j in range(DF.shape[0]):
        if '- 00:00' in DF.iloc[j,0]:
            index = j
            DF = DF.iloc[:index+1,:]
            break
    return DF

def check_df_2(DF):
    for j in range(DF.shape[0]):
        if 'Mud Log Information' in DF.iloc[j,0] or "06:00 Update" in DF.iloc[j,0]:
            index = j
            DF = DF.iloc[:index,:]
            break
    return DF

def row_check(DF):
    lst = []
    for i in range(len(DF)):
        if "Operations Summary" in DF.iloc[i,0] or "From" in DF.iloc[i,0]:
            lst.append(i)

    DF.reset_index(inplace=True)
    DF.drop(columns = "index",axis=1,inplace=True)
    # ind = row_check(all)
    DF.drop(lst,inplace=True)  
    return DF

def get_df(pdf):
    updated_df_i = pd.DataFrame()
    all_df = pd.DataFrame()
    for i in range(len(pdf.pages)):
        #print(i)
        page = pdf.pages[i]
        lst = page.extract_table()
        df_i = pd.DataFrame(lst, columns=lst[0])
        df_i = df_i.fillna(value="")
        #df_i = df_i.dropna(axis=0, how='any')
        
        for k in range(df_i.shape[0]):
            if "Operations Summary" in df_i.iloc[k,0]:
                ind = k

                updated_df_i = df_i.iloc[ind+1:,:]
                updated_df_i = check_df_1(updated_df_i)
                updated_df_i = check_df_2(updated_df_i)


                updated_df_i = updated_df_i.dropna(axis=1)
                headers = updated_df_i.iloc[0]
                updated_df_i  = pd.DataFrame(updated_df_i.values[1:], columns=headers)
                updated_df_i = columns_check(updated_df_i)
                
            
                all_df = all_df.append(updated_df_i)
                all_df = all_df.reset_index()
                all_df.drop(columns = "index",axis=1,inplace=True)
                break
            
            else:
                pass

    
    return all_df.drop_duplicates()

In [159]:
## One major function to append all DFs

def get_all_data(root):

    final_df = pd.DataFrame()
    for path in list_dir(root):
        print(path)
        pdf = pdfplumber.open(path)
        #all_tables = read_pdf(path,pages="all")
        operations_summary = get_df(pdf)
        final_df = pd.concat([final_df,operations_summary])
        print(final_df.shape)

    final_df = row_check(final_df)

    return final_df 

In [1]:
all = get_all_data("/Users/angshuman.chakraborty@cognitedata.com/Desktop/work/BP/DDRs_samples_BP_GoM/2017") ## folder path

## Cleaning and Preprocessing
<b>Operations summary: Cleaning and Preparation</b>


In [379]:
DF = pd.read_csv("/Users/angshuman.chakraborty@cognitedata.com/Desktop/work/BP/DDRs/all2021.csv")
pd.set_option('display.max_colwidth', None)
df = DF.copy()

In [439]:
def preprocessing(df):
    df.drop(['Unnamed: 0'], axis = 1, inplace = True)
    df.replace({'\n' : ''}, regex = True, inplace = True)
    df.columns = [col[:19] if '(ft)' in col else col for col in df.columns]
    df.columns = [col[:19] if '(usft)' in col else col for col in df.columns]
    df = df.rename(columns = {'From - To\nOp. Depth' : 'From - To Op. Depth',
                            'Hrs\n(hr)' : 'Hrs (hr)',
                            'Critical \nPath' : 'Critical Path'})
    

    df['From - To Op. Depth'] = df['From - To Op. Depth'].astype('str')
    df['From - To'] = [value[:13] for value in df['From - To Op. Depth']]
    df['Depth (usft)'] = [value[14:] for value in df['From - To Op. Depth']]
    df.drop(['From - To Op. Depth'], axis = 1, inplace = True)
    
    df = df.fillna(' ')
    order = [8,9,0,1,2,3,4,5,6,7] # setting column's order
    new_df = df[[df.columns[i] for i in order]]
    
    return new_df

In [440]:
#Removes and merges white spaces and splits rows into multiple rows based on activity
def removeSpaces(df):
    columns = [col for col in df.columns if col != 'Operation' and col != 'From - To']
    indices = []
    for i in range(len(df)):
        j = 0
        for col in columns:
            if df.loc[i, col] == ' ':
                j += 1
        if j == len(columns) - 1:
            indices.append(i)
    for index in indices:
        df.loc[index - 1, 'Operation'] = df.loc[index - 1, 'Operation'] + ' ' + df.loc[index, 'Operation']
    df = df.drop(indices)
    df.reset_index(inplace = True, drop = True)
    
    df = df.assign(Operation=df.Operation.str.split('. -')).explode('Operation') ## this could be document/well specific - needs to be modified probably
    df.reset_index(inplace = True, drop = True)
    return df

In [443]:
# Text Preprocessing
import string
import spacy
nlp = spacy.load('en_core_web_sm')
from html import unescape


def processing_sentence(df):

    # Remove Punctuations
    punc_str = '!#$%&\*+,-:;<=>?@[\\]^_`{|}~'
    df['processed_operation'] = df['Operation'].apply(lambda x: "".\
                                                            join([char for char in x if char not in punc_str]))

    # # Lower case all the words: Not necessary?
    # df['processed_operation'] = df['Operation'].apply(lambda x : x.lower())
    # df['processed_operation'] = df['Operation'].str.replace("[\d-]",'',regex=True)

    # Removal of stopwords
    stopwords = nlp.Defaults.stop_words
    def remove_stopwords(text):
        output= [i for i in text if i not in stopwords]
        return output

    df['processed_operation'] = df['processed_operation'].apply(lambda x:remove_stopwords(x))
    df['processed_operation'] = df['processed_operation'].apply(lambda x: "".join(x))

    return df

In [3]:
df = preprocessing(df)
df = removeSpaces(df)
df = processing_sentence(df)
